# Sigma-Lab — Stress Test & Interactive Demo (v4.2)

This notebook helps you **exercise** the Sigma-Lab framework:
- Run the built-in **demo contexts** (`healthcare`, `ai`, `public`).
- Generate **random scenarios** to probe stability.
- Visualize **score distributions** and **equity metrics**.
- Export a compact **audit sample** for documentation.

> **Prereq:** place `sigma_lab_v4_2.py` in the same folder as this notebook.

In [ ]:
import json, random, statistics as stats
import matplotlib.pyplot as plt

from sigma_lab_v4_2 import SigmaLab, demo_context, OptionContext

In [ ]:
def run_demo(domain: str="healthcare", pretty: bool=True):
    cfg, ctx = demo_context(domain)
    engine = SigmaLab(cfg)
    result = engine.diagnose(ctx, verdict_opt_in=True)
    if pretty:
        print(json.dumps(result, indent=2, ensure_ascii=False))
    return engine, ctx, result

# Quick smoke test (no pretty dump)
engine, ctx, result = run_demo("healthcare", pretty=False)
print("Demo engine ready. Keys:", list(result.keys()))

### Randomized scenarios

In [ ]:
def random_context(i:int=0) -> OptionContext:
    # Inputs will be clamped by Sigma-Lab; we allow a slightly wider range for stress.
    return OptionContext(
        name=f"rnd_{i}",
        short_term_risk=random.uniform(0, 1.1),
        long_term_risk=random.uniform(0, 1.1),
        irreversibility_risk=random.uniform(0, 1.0),
        stakeholders=[
            dict(name="minority_users", vulnerability=random.uniform(0.2,0.9), impact_benefit=random.uniform(0.3,0.8)),
            dict(name="majority_users",  vulnerability=random.uniform(0.1,0.6), impact_benefit=random.uniform(0.4,0.9)),
            dict(name="operators",      vulnerability=random.uniform(0.1,0.5), impact_benefit=random.uniform(0.3,0.8)),
        ]
    )


def sweep(n=200, domain="ai"):
    cfg, _ = demo_context(domain)
    eng = SigmaLab(cfg)
    rows = []
    for i in range(n):
        ctx = random_context(i)
        out = eng.diagnose(ctx, verdict_opt_in=True)
        scores = out.get("scores", {})
        rows.append({
            "i": i,
            "non_harm":  scores.get("non_harm", 0.0),
            "stability":  scores.get("stability", 0.0),
            "resilience": scores.get("resilience", 0.0),
            "equity":     scores.get("equity", 0.0),
            "verdict": (out.get("semantics",{}) or {}).get("verdict_procedural",{}).get("status","N/A")
        })
    return rows

rows = sweep(n=200, domain="ai")
len(rows), rows[0]

### Visualize distributions

In [ ]:
def plot_hist(values, title):
    plt.figure(figsize=(6,4))
    plt.hist(values, bins=20)
    plt.title(title)
    plt.xlabel("Score")
    plt.ylabel("Frequency")
    plt.show()

nh = [r["non_harm"]  for r in rows]
st = [r["stability"] for r in rows]
re = [r["resilience"] for r in rows]
eq = [r["equity"]    for r in rows]

plot_hist(nh, "Non-Harm score distribution")
plot_hist(st, "Stability score distribution")
plot_hist(re, "Resilience score distribution")
plot_hist(eq, "Equity score distribution")

### Verdict summary

In [ ]:
from collections import Counter
verdicts = Counter([r["verdict"] for r in rows])
print("Verdict counts:", dict(verdicts))
print("Non-Harm avg:", round(stats.mean(nh),3), "±", round(stats.pstdev(nh),3))

### Export a compact audit sample

In [ ]:
import time, json
cfg, ctx = demo_context("public")
engine = SigmaLab(cfg)
res = engine.diagnose(ctx, verdict_opt_in=True)
audit = {
    "timestamp": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime()),
    "context_name": ctx.name,
    "scores": res.get("scores",{}),
    "veto": res.get("veto",{}),
    "provenance": res.get("provenance",{}),
}
with open("audit_sample_public.json", "w", encoding="utf-8") as f:
    json.dump(audit, f, ensure_ascii=False, indent=2)
print("Saved: audit_sample_public.json")